In [2]:
import random
import numpy as np
import numpy.matlib
import pandas as pd

feature_i = pd.read_csv("p2/feature_i.csv", index_col=0)
feature_i = feature_i.loc[feature_i["day"]>=7,].reset_index()
del feature_i["index"]
feature_i

,pluno,bndno,type5,type4,type3,type2,day,weekday,qty,d-1,d-2,d-3,d-4,d-5,d-6,d-7
0,22000005,22000005,22000,2200,220,22,7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22000005,22000005,22000,2200,220,22,8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22000005,22000005,22000,2200,220,22,9,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22000005,22000005,22000,2200,220,22,10,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22000005,22000005,22000,2200,220,22,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106745,27421002,27421002,27421,2742,274,27,177,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106746,27421002,27421002,27421,2742,274,27,178,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106747,27421002,27421002,27421,2742,274,27,179,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106748,27421002,27421002,27421,2742,274,27,180,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#i.  training 7-146 svm
x_i = feature_i.loc[(feature_i["day"] <= 146), ["pluno", "bndno", "type5", "type4", "type3", "type2", "day", "weekday", "d-1", "d-2", "d-3", "d-4", "d-5", "d-6", "d-7"]].values
y_i = feature_i.loc[(feature_i["day"] <= 146),"qty"].values
x_i = np.reshape(x_i, (-1, 15))
y_i = np.reshape(y_i, (-1, 1))

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_i = sc_X.fit_transform(x_i)
Y_i = sc_y.fit_transform(y_i)

In [4]:
import time
time_list = [[0,0],[0,0],[0,0]]

from sklearn.svm import SVR
time_start=time.time()
regressor_i = SVR(kernel = 'rbf')
regressor_i.fit(X_i, Y_i)
time_end=time.time()
time_list[0][0] = time_end - time_start

test_i = feature_i.loc[(feature_i["day"] > 146),:].reset_index()
time_start=time.time()
for i, row in test_i.iterrows():
    j = 1
    while j <= row["day"]%7:
        test_i.loc[i, "d-"+str(j)] = test_i.loc[i-j, "predict"]
        j = j+1
    y_pred = regressor_i.predict(
        sc_X.transform(
            np.reshape(
            row[["pluno", "bndno", "type5", "type4", "type3", "type2", "day", "weekday", "d-1", "d-2", "d-3", "d-4", "d-5", "d-6", "d-7"]].values
            , (-1, 15))))
    test_i.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
# test_i.loc[, ["day", "qty", "predict"]]
time_end=time.time()
time_list[0][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
# i. random forest
from sklearn.ensemble import RandomForestRegressor 
time_start=time.time()
rfr_i =RandomForestRegressor()
rfr_i.fit(X_i, Y_i)
time_end=time.time()
time_list[1][0] = time_end - time_start

test_i_rfr = feature_i.loc[(feature_i["day"] > 146),:].reset_index()
time_start=time.time()
for i, row in test_i_rfr.iterrows():
    j = 1
    while j <= row["day"]%7:
        test_i_rfr.loc[i, "d-"+str(j)] = test_i_rfr.loc[i-j, "predict"]
        j = j+1
    y_pred = rfr_i.predict(
            sc_X.transform(
            np.reshape(
            row[["pluno", "bndno", "type5", "type4", "type3", "type2", "day", "weekday", "d-1", "d-2", "d-3", "d-4", "d-5", "d-6", "d-7"]].values
            , (-1, 15))))
    test_i_rfr.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i_rfr
time_end=time.time()
time_list[1][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [6]:
# i. MLP
from sklearn.neural_network import MLPRegressor
time_start=time.time()
mlp_i = MLPRegressor(
        hidden_layer_sizes=(100,50), activation='relu',solver='adam',
        alpha=0.001,max_iter=200)
# mlp_i = MLPRegressor()
mlp_i.fit(X_i,Y_i)
time_end=time.time()
time_list[2][0] = time_end - time_start

test_i_mlp = feature_i.loc[(feature_i["day"] > 146),:].reset_index()
time_start=time.time()
for i, row in test_i_mlp.iterrows():
    j = 1
    while j <= row["day"]%7:
        test_i_mlp.loc[i, "d-"+str(j)] = test_i_mlp.loc[i-j, "predict"]
        j = j+1
    y_pred = mlp_i.predict(
        sc_X.transform(
            np.reshape(
            row[["pluno", "bndno", "type5", "type4", "type3", "type2", "day", "weekday", "d-1", "d-2", "d-3", "d-4", "d-5", "d-6", "d-7"]].values
            , (-1, 15))))
    test_i_mlp.loc[i, "predict"] = sc_y.inverse_transform(y_pred)
test_i_mlp
time_end=time.time()
time_list[2][1] = time_end - time_start

C:\Users\admin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
import math

hat_svr = test_i.loc[:, ["qty", "predict", "day"]]
hat_svr.to_csv("2p3/1/svr.csv")
avg_y_svr = np.mean(hat_svr["qty"])
RSE_svr = math.sqrt((hat_svr["qty"]-hat_svr["predict"]).apply(lambda x:math.pow(x, 2)).sum()/hat_svr["qty"].apply(lambda x:math.pow(x-avg_y_svr, 2)).sum())
RSE_svr

0.9976846464361963

In [14]:
hat_rfr = test_i_rfr.loc[:, ["qty", "predict", "day"]]
hat_rfr.to_csv("2p3/1/rfr.csv")
avg_y_rfr = np.mean(hat_rfr["qty"])
RSE_rfr = math.sqrt((hat_rfr["qty"]-hat_rfr["predict"]).apply(lambda x:math.pow(x, 2)).sum()/hat_rfr["qty"].apply(lambda x:math.pow(x-avg_y_rfr, 2)).sum())
RSE_rfr

0.9009606998114897

In [15]:
hat_mlp = test_i_mlp.loc[:, ["qty", "predict", "day"]]
hat_mlp.to_csv("2p3/1/mlp.csv")
avg_y_mlp = np.mean(hat_mlp["qty"])
RSE_mlp = math.sqrt((hat_mlp["qty"]-hat_mlp["predict"]).apply(lambda x:math.pow(x, 2)).sum()/hat_mlp["qty"].apply(lambda x:math.pow(x-avg_y_mlp, 2)).sum())
RSE_mlp

1.0703638155102346

In [11]:
def getRSE(data):
    avg = np.mean(data["qty"])
    return math.sqrt((data["qty"]-data["predict"]).apply(lambda x:math.pow(x, 2)).sum()/data["qty"].apply(lambda x:math.pow(x-avg, 2)).sum())

In [16]:
horizon = range(1, 8)
results = [hat_svr, hat_rfr, hat_mlp]
RES_results = []
for re in results:
    ls = []
    for i in horizon:
        ls.append(getRSE(re.loc[re["day"]%7==(i-1), :]))
    RES_results.append(ls)
RES_results

[[0.99741122896008,
  1.0003472368488218,
  0.9973578099592385,
  1.007444743667932,
  0.9972405799275884,
  0.995673836231623,
  0.9963844935142236],
 [0.43033914434659304,
  0.8711150785892618,
  0.8505587545304785,
  1.7321745015309533,
  0.9916879291683497,
  0.8164856083786179,
  1.0148171013402523],
 [1.0158304970355212,
  1.0137003559979647,
  0.877483576253748,
  2.373460402654206,
  0.9749796649623728,
  0.8045087127775142,
  0.9733096485164573]]

In [17]:
time_list

[[240.43615651130676, 34.77296686172485],
 [43.087199687957764, 125.17625713348389],
 [24.76494574546814, 37.07049322128296]]